In [ ]:
# Dash -8 


!python train.py --data coco.yaml --cfg yolov5s.yaml --weights '' --batch-size 64

In [4]:
import os
day_folders = os.listdir('/media/data1/DASHLAB_FALLDOWN_cropped/data/day_night_split/day')
print(day_folders)

['GOPR1798', 'GOPR2691', 'GOPR2700', 'GOPR2705', 'MVI_0002', 'GOPR2698', 'GOPR1874', 'GOPR1693', 'GOPR2726', 'GOPR1794', 'GOPR1858', 'GOPR2687', 'GOPR1864', 'GOPR1797', 'GOPR2096', 'GOPR1805', 'GOPR0830', 'GOPR1789', 'GOPR1876', 'GOPR2150', 'GOPR2727', 'GOPR2701', 'GOPR1857', 'GOPR2693', 'GOPR1855', 'GOPR1781', 'GOPR2692', 'GOPR2669', 'GOPR2666', 'GOPR1810', 'GOPR1808', 'GOPR1878', 'GOPR1866', 'GOPR2708', 'GOPR0806', 'GOPR1877', 'GOPR2711', 'GOPR0808', 'MVI_0008', 'GOPR2456', 'GOPR2664', 'GOPR2036', 'GOPR0829', 'GOPR0807', 'GOPR1860', 'GOPR2712', 'GOPR2342', 'GOPR2735', 'GOPR0863', 'MVI_0010', 'GOPR0827', 'GOPR2662', 'GOPR0815', 'GOPR1861', 'GOPR2695', 'GOPR2681', 'GOPR2663', 'GOPR2678', 'GOPR0860', 'GOPR0864', 'GOPR1873', 'MVI_0009', 'GOPR0859', 'GOPR0802', 'GOPR2422', 'GOPR1813', 'GOPR2676', 'GOPR2710', 'GOPR0824', 'GOPR1859', 'GOPR1790', 'GOPR2670', 'GOPR2672', 'GOPR1879', 'GOPR2732', 'GOPR1796', 'GOPR2157', 'GOPR0865', 'GOPR2728', 'GOPR0951', 'GOPR2673', 'GOPR0805', 'GOPR1875', 'GO

In [5]:
import sys
import os
import csv

import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree
from PIL import Image

import json

from xml.etree.ElementTree import dump

In [8]:
json_path = '/media/data1/DASHLAB_FALLDOWN_cropped/data/day_night_split/annotations/day.json'
json_data = json.load(open(json_path))

In [24]:
images_info = json_data["images"]
type_info = json_data["type"]
cls_info = json_data["categories"]
anno_info = json_data["annotations"]

In [30]:
print(type_info)
print(cls_info)

instances
[{'supercategory': 'none', 'id': 1, 'name': '0'}, {'supercategory': 'none', 'id': 2, 'name': '1'}]


In [29]:
print(images_info[0])
print(anno_info[0])

{'file_name': 'MVI_0007/images/MVI_0007_01083.jpg', 'height': 1080, 'width': 1920, 'id': 'MVI_0007/images/MVI_0007_01083'}
{'area': 271050, 'iscrowd': 0, 'bbox': [1018, 242, 325, 834], 'category_id': 1, 'ignore': 0, 'segmentation': [], 'image_id': 'MVI_0007/images/MVI_0007_01083', 'id': 1}


In [31]:
print(len(images_info))
print(len(anno_info))

46282
186140


In [36]:
import tqdm

data = {}

for anno in tqdm.tqdm(anno_info):
    img_id = anno['image_id']
    cls_id = anno['category_id']
    
    filename = None
    img_width = None
    img_height = None
    cl = None
    
    for info in images_info:
        if info['id'] == img_id:
            filename = info['id']
            img_width = info['width']
            img_height = info['height']
    
    for cat in cls_info:
        if cat['id'] == cls_id:
            cl = cat['name']
        
    size = {
        'width': img_width,
        'height': img_height
    }
    
    bbox = {
        "xmin": anno["bbox"][0],
        "ymin": anno["bbox"][1],
        "xmax": anno["bbox"][2] + anno["bbox"][0],
        "ymax": anno["bbox"][3] + anno["bbox"][1]
    }
    
    obj_info = {
        'name': cl,
        'bndbox': bbox
    }
    
    if filename in data:
        obj_idx = str(int(data[filename]["objects"]["num_obj"]))
        data[filename]["objects"][str(obj_idx)] = obj_info
        data[filename]["objects"]["num_obj"] = int(obj_idx) + 1
    elif filename not in data:
        obj = {
            "num_obj": "1",
            "0": obj_info
        }

        data[filename] = {
            "size": size,
            "objects": obj,
            'filename': filename
        }

100%|██████████| 186140/186140 [10:13<00:00, 303.62it/s]


In [74]:
path = '/media/data1/DASHLAB_FALLDOWN_cropped/data/day_night_split/day'

for img in tqdm.tqdm(data):
    label_path = os.path.join(path, img.replace('images','labels')+'.txt')
    label_folder = '/'.join(label_path.split('/')[:-1])
    if not os.path.isdir(label_folder):                                                           
        os.mkdir(label_folder)
    ff = open(label_path, 'w')
    ff.close()
    width, height = data[img]['size']['width'], data[img]['size']['height']
    for i in range(len(data[img]['objects'])-1):
        
        cl = data[img]['objects'][str(i)]['name']
        xmin, ymin = data[img]['objects'][str(i)]['bndbox']['xmin'], data[img]['objects'][str(i)]['bndbox']['ymin']
        xmax, ymax = data[img]['objects'][str(i)]['bndbox']['xmax'], data[img]['objects'][str(i)]['bndbox']['ymax']
        
        x = (xmin + xmax) / 2
        y = (ymin + ymax) / 2
        w = xmax - xmin
        h = ymax - ymin
        
        ff = open(label_path, 'a')
        ff.write(str(cl) + ' ' + str(x/width) + ' ' + str(y/height) + ' ' + str(w/width) + ' ' + str(h/height) + '\n')
        ff.close()

100%|██████████| 45691/45691 [00:05<00:00, 8174.12it/s]
